In [ ]:
from scipy.io.wavfile import write
import sounddevice as sd
import queue
import threading
import numpy as np
from bark import SAMPLE_RATE, generate_audio, preload_models
from transformers import pipeline
import IPython.display as ipd

# Load Bark models
preload_models()

# Define a queue to hold audio data
audio_queue = queue.Queue()

# Define a callback function to capture audio
def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

# Function to convert audio to text using a speech-to-text model
def audio_to_text(audio_data):
    stt_pipeline = pipeline("automatic-speech-recognition")
    transcription = stt_pipeline(audio_data)
    return transcription['text']

# Function to generate audio from text using Bark
def text_to_audio(text):
    audio_array = generate_audio(text)
    return audio_array

# Function to play audio
def play_audio(audio_array):
    sd.play(audio_array, samplerate=SAMPLE_RATE)
    sd.wait()

# Thread function to process the audio
def process_audio():
    while True:
        if not audio_queue.empty():
            audio_data = audio_queue.get()
            # Convert audio data to text
            text = audio_to_text(audio_data)
            # Generate audio from text
            generated_audio = text_to_audio(text)
            # Play the generated audio
            play_audio(generated_audio)

# Start audio stream
stream = sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE)
stream.start()

# Start processing thread
audio_thread = threading.Thread(target=process_audio)
audio_thread.start()

# Display a message indicating that the real-time voice changer is running
print("Real-time voice changer is running...")

# Since the notebook is running a real-time application, it will not terminate on its own.
# The cell will keep running, allowing continuous audio processing.
# To stop the execution, you will need to interrupt the kernel.